In [1]:
import pandas as pd
import numpy as np
import pickle
import re
from tqdm import tqdm


import shap
from plotly.offline import init_notebook_mode
import plotly.offline as py
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt

In [14]:
df = pd.read_csv("../data/datasets/toxic_comment/train.csv")
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print('Size: ', len(df))
print(df[classes].sum() / len(df) * 100)

Size:  159571
toxic            9.584448
severe_toxic     0.999555
obscene          5.294822
threat           0.299553
insult           4.936361
identity_hate    0.880486
dtype: float64


In [15]:
df = pd.read_csv("../data/datasets/hate_abuse_speech/hatespeech_text_label_vote.csv", sep='\t', header=None, names=['text', 'label', 'votes'])
print('Size: ', len(df))
print(df.groupby('label')['label'].count() / len(df) * 100)

Size:  99996
label
abusive    27.151086
hateful     4.965199
normal     53.853154
spam       14.030561
Name: label, dtype: float64


In [16]:
df = pd.read_csv("../data/datasets/hatespeech_offensive/labeled_data.csv")
print('Size: ', len(df))
print(df.groupby('class')['class'].count() / len(df) * 100)

Size:  24783
class
0     5.770084
1    77.432111
2    16.797805
Name: class, dtype: float64


In [21]:
df = pd.read_csv("../data/datasets/online_harassment_dataset/onlineHarassmentDataset.tdf", sep='\t', encoding='latin-1', memory_map=True)
df['Label'] = df['Code'].apply(lambda x: 1 if x == 'H' else 0)
df = df[['ID', 'Label', 'Tweet']]
print('Size: ', len(df))
print(df.groupby('Label')['Label'].count() / len(df) * 100)

Size:  20360
Label
0    74.04224
1    25.95776
Name: Label, dtype: float64


In [20]:
df

,ID,Label,Tweet
0,1,1,@Grumpy_P_Sloth @deanesmay feminists argue for...
1,2,1,1948Army of England helped the Jews to occupy ...
2,3,1,Dutch Leader Says Europe to collapse In 6 Week...
3,4,1,RT @__DeLay: The next day the Romans and the J...
4,5,1,RT @Bakersman_Joe: When Hitler Invited The Jew...
...,...,...,...
20355,20356,1,Would you have intervened to arm the Jews to o...
20356,20357,1,Would you like me to release to you this King ...
20357,20358,1,You Fucking Nigger You Did It To Yourself You ...
20358,20359,1,Young white woman gunned down by black beast i...


In [ ]:
def get_toxic_comments_dataset():
    # From PerspectiveAI Kaggle competition
    df = pd.read_csv("../data/datasets/toxic_comment/train.csv")
    classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    Y = df[classes].astype(int).values
    X = df['comment_text']
    return X, Y, classes, df

# def get_hate_abuse_dataset():
#     # From: Large Scale Crowdsourcing and Characterization of Twitter Abusive Behavior
#     df = pd.read_csv("../data/datasets/hate_abuse_speech/hatespeech_text_label_vote.csv", sep='\t', header=None, names=['text', 'label', 'votes'])
#     X = df['text']
#     Y = df['label']
#     classes = np.unique(Y)
#     return X, Y.values, classes, df


def get_hate_offensive_speech_dataset():
    # Automated Hate Speech Detection and the Problem of Offensive Language∗
    df = pd.read_csv("../data/datasets/hatespeech_offensive/labeled_data.csv")
    X = df['tweet']
    Y = df['class']
    classes = ['hate', 'offensive', 'neither']
    return X, Y, classes, df


def get_harassment_umd_dataset():
    df = pd.read_csv("../data/datasets/online_harassment_dataset/onlineHarassmentDataset.tdf", sep='\t', encoding='latin-1', memory_map=True)
    df['Label'] = df['Code'].apply(lambda x: 1 if x == 'H' else 0)
    df = df[['ID', 'Label', 'Tweet']]
    X = df['Tweet']
    Y = df['Label'].astype(int)
    classes = ['Harassment', 'Normal']
    return X, Y, classes, df